In [1]:
import pandas as pd
import numpy as np
import copy
import math
from datetime import datetime
import time
from urllib.parse import quote_plus
from sqlalchemy import create_engine
import sqlalchemy as sa

In [2]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_columns', None)

In [164]:
passes = pd.read_csv(r"C:\Users\vsaburov\Airlines\boarding_passes_upd.csv", parse_dates=['date'])

In [169]:
passes.nunique()

sequence           101
gender               2
name            229247
agency           83529
flight           23950
from              1093
to                1092
airport_from      1128
airport_to        1127
date               366
time              1197
pnr             615146
TicketNumber    615771
class                4
dtype: int64

In [170]:
joined = pd.read_csv(r"C:\Users\vsaburov\Airlines\joined.csv", parse_dates=['PassengerBirthDate', 'DepartureDate'])

In [171]:
joined

,ArrivalCity,BaggageState,BookingCode,CodeShare,DepartureDate,DepartureTime,FlightNumber,PassengerBirthDate,PassengerDocument,PassengerFirstName,PassengerLastName,PassengerSecondName,PassengerSex,TicketNumber,AgentInfo,ArrivalAirport,ArrivalDate,ArrivalTime,BaggageCount,DepartureAirport,Fare,Meal,PassengerAdditionalInfo,PassengerClass,ArrivalCountry,DepartureCity,DepartureCountry,PassengerNickName
0,Moscow,Transit,WMXPVY,False,2017-01-01,09:15,SU1001,1999-12-13,5140 683757,ALISA,ABRAMOVA,R.,Female,4.178367e+15,Travelgenio,SVO,2017-01-01,12:10,1PC,KGD,YFLXYW,RVML,F,Y,NaN,NaN,NaN,NaN
1,NaN,NaN,ULCPQH,False,2017-01-01,09:15,SU1001,NaT,7477 017451,KRISTINA,ARTEMOVA,MARATOVNA,NaN,7.935022e+14,OneTwoTrip,SVO,2017-01-01,12:10,NaN,KGD,YFLXEY,NaN,NaN,Y,NaN,NaN,NaN,NaN
2,NaN,NaN,YVICRN,False,2017-01-01,09:15,SU1001,NaT,6840 635650,ALEKSANDR,BABUSHKIN,ALEKSANDROVICH,NaN,2.360894e+15,eDreams,SVO,2017-01-01,12:10,1PC,KGD,YSTNUB,NaN,4 FF#FB 483772175,Y,NaN,NaN,NaN,NaN
3,Moscow,Registered,NaN,False,2017-01-01,09:15,SU1001,1974-07-09,1205 586135,MARK,BEREZIN,A.,Male,8.200536e+15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,DBEVCU,False,2017-01-01,09:15,SU1001,1990-05-04,6601 454145,VASILISA,BLINOVA,ILDAROVNA,NaN,7.315197e+15,City.Travel,SVO,2017-01-01,12:10,1PC,KGD,YGRPPY,VGML,0 FF#SU 806242829,Y,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239712,NaN,NaN,VERHMP,False,2018-01-01,09:55,SU6183,NaT,2941 115924,EGOR,LEVIN,SAVELEVICH,NaN,2.038332e+15,City.Travel,ROV,2018-01-01,11:55,NaN,SVO,YSTNII,MOML,#,Y,NaN,NaN,NaN,NaN
239713,Moscow,None,DWPNNZ,False,2018-01-01,12:45,SU6184,1978-03-30,2191 291628,ANZHELIKA,BLINOVA,MAKAREVNA,Female,6.149777e+15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
239714,Moscow,Registered,RWUBBA,False,2018-01-01,16:30,SU6286,1973-05-01,2022 416117,ALEKSANDRA,KORCHAGINA,GLEBOVNA,Female,1.650099e+15,NaN,SVO,2018-01-01,16:05,NaN,PKC,YFLXGK,STML,K FF#KE 978286566,Y,NaN,NaN,NaN,NaN
239715,Khabarovsk,Registered,STCVTK,False,2018-01-01,17:40,SU6287,1995-04-28,6815 172060,SVETLANA,ZLOBINA,D.,Female,2.360753e+15,Aerobilet,KHV,2018-01-02,08:15,NaN,SVO,YRSTQE,NaN,U FF#SU 316501612,Y,NaN,NaN,NaN,NaN


In [12]:
joined[joined.isna()==True]

,ArrivalCity,BaggageState,BookingCode,CodeShare,DepartureDate,DepartureTime,FlightNumber,PassengerBirthDate,PassengerDocument,PassengerFirstName,PassengerLastName,PassengerSecondName,PassengerSex,TicketNumber,AgentInfo,ArrivalAirport,ArrivalDate,ArrivalTime,BaggageCount,DepartureAirport,Fare,Meal,PassengerAdditionalInfo,PassengerClass,ArrivalCountry,DepartureCity,DepartureCountry,PassengerNickName
0,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239712,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
239713,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
239714,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
239715,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
aa = joined['BookingCode'].value_counts().loc[lambda x : x > 1].index

In [21]:
#.query("BookingCode=='AJUDMG & ")

,ArrivalCity,BaggageState,BookingCode,CodeShare,DepartureDate,DepartureTime,FlightNumber,PassengerBirthDate,PassengerDocument,PassengerFirstName,PassengerLastName,PassengerSecondName,PassengerSex,TicketNumber,AgentInfo,ArrivalAirport,ArrivalDate,ArrivalTime,BaggageCount,DepartureAirport,Fare,Meal,PassengerAdditionalInfo,PassengerClass,ArrivalCountry,DepartureCity,DepartureCountry,PassengerNickName
75613,NaN,NaN,AADBRW,False,2017-01-22,16:40,SU1465,NaT,2017 682831,IURII,TIHOMIROV,RUSLANOVICH,NaN,6.188689e+15,OZON.travel,SVO,2017-01-22,18:15,NaN,SKX,YFLXKU,NaN,NaN,Y,NaN,NaN,NaN,NaN
75614,Moscow,None,AADBRW,False,2017-01-22,16:40,SU1465,1985-02-09,2017 682831,IURII,TIKHOMIROV,R.,Male,6.188689e+15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86332,Khabarovsk,None,AAKHLR,True,2017-01-26,12:10,SU4606,1990-06-27,6690 539911,SAMIR,TIKHONOV,SAVVOVICH,Male,5.123915e+15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86331,NaN,NaN,AAKHLR,True,2017-01-26,12:10,SU4606,NaT,6690 539911,SAMIR,TIHONOV,SAVVOVICH,NaN,5.123915e+15,Aerobilet,KHV,2017-01-26,13:20,NaN,OHH,YGRPYZ,NLML,D FF#DT 902110667,Y,NaN,NaN,NaN,NaN
220535,Khabarovsk,Delayed,AAKIKG,False,2017-08-13,16:45,SU5685,1999-05-18,3591 237275,ZAKHAR,SEVASTIANOV,V.,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148280,NaN,NaN,ZZLRLO,True,2017-03-08,13:50,SU4606,1998-10-28,2251 558414,VASILISA,MALAHOVA,VLADISLAVOVNA,NaN,5.462556e+15,OZON.travel,OHH,2017-03-08,15:00,NaN,KHV,YGRPGU,NaN,#,Y,NaN,NaN,NaN,NaN
34564,NaN,NaN,ZZSJIQ,False,2017-01-08,06:40,SU1383,1975-05-22,5089 708639,DMITRII,ALEHIN,AIDAROVICH,NaN,9.044572e+15,KupiBilet,SVO,2017-01-08,07:55,1PC,HMA,YRSTDN,NaN,1 FF#FB 343941148,Y,NaN,NaN,NaN,NaN
34565,Moscow,Delayed,ZZSJIQ,False,2017-01-08,06:40,SU1383,1975-05-22,5089 708639,DMITRII,ALEKHIN,AIDAROVICH,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106495,Moscow,None,ZZXAIM,False,2017-02-06,09:55,SU1443,1990-06-26,3322 406270,ALENA,VERESHCHAGINA,MIROSLAVOVNA,Female,NaN,NaN,SVO,NaN,NaN,NaN,IKT,NaN,NaN,NaN,NaN,Russian Federation,Irkutsk,Russian Federation,FrequentFlyer7123


In [20]:
joined[joined['TicketNumber'].isna()]

,ArrivalCity,BaggageState,BookingCode,CodeShare,DepartureDate,DepartureTime,FlightNumber,PassengerBirthDate,PassengerDocument,PassengerFirstName,PassengerLastName,PassengerSecondName,PassengerSex,TicketNumber,AgentInfo,ArrivalAirport,ArrivalDate,ArrivalTime,BaggageCount,DepartureAirport,Fare,Meal,PassengerAdditionalInfo,PassengerClass,ArrivalCountry,DepartureCity,DepartureCountry,PassengerNickName
6,Moscow,Transit,QVSCOZ,False,2017-01-01,09:15,SU1001,1977-04-27,9115 247669,VALENTINA,DEMIANOVA,T.,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,Moscow,None,NaN,False,2017-01-01,09:15,SU1001,1974-12-19,4344 056129,RADMIR,TITOV,A.,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,Moscow,Registered,NaN,False,2017-01-01,12:45,SU1003,1977-06-14,9059 940182,VALENTINA,SOROKINA,M.,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,Moscow,None,NaN,False,2017-01-01,16:00,SU1005,1996-07-18,7672 731116,KONSTANTIN,KALININ,T.,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55,Moscow,Delayed,NaN,False,2017-01-01,19:55,SU1007,1985-02-03,2843 024798,EVA,AGEEVA,A.,Female,NaN,NaN,SVO,NaN,NaN,NaN,KGD,NaN,NaN,NaN,NaN,Russian Federation,Kaliningrad,Russian Federation,FrequentFlyer71659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239693,Moscow,Delayed,SPNIZY,False,2018-01-01,05:35,SU1763,1971-10-11,6757 413317,DAMIR,UVAROV,M.,Male,NaN,NaN,SVO,NaN,NaN,NaN,OMS,NaN,NaN,NaN,NaN,Russian Federation,Omsk,Russian Federation,FrequentFlyer72564
239695,Nogliki,Registered,NaN,True,2018-01-01,09:00,SU4599,1978-10-08,7852 822707,AMINA,ZAKHAROVA,A.,Female,NaN,NaN,NGK,NaN,NaN,NaN,KHV,NaN,NaN,NaN,NaN,Russian Federation,Khabarovsk,Russian Federation,FrequentFlyer65362
239699,Vladivostok,Transit,CVRBDQ,False,2018-01-01,19:00,SU5601,1971-02-09,5404 428321,SVIATOGOR,KOCHETOV,V.,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
239704,Khabarovsk,Registered,VLLSEM,False,2018-01-01,05:40,SU5618,1974-12-28,8772 037689,MIROSLAVA,LAPTEVA,ANATOLEVNA,Female,NaN,NaN,KHV,NaN,NaN,NaN,VVO,NaN,NaN,NaN,NaN,Russian Federation,Vladivostok,Russian Federation,FrequentFlyer33526


In [22]:
s1 = joined[joined['BookingCode'].isin(aa)].sort_values('BookingCode')
s2 = joined[joined['BookingCode'].isin(aa)].sort_values('BookingCode')

In [23]:
u = s1.merge(s2, left_on = 'BookingCode', right_on = 'BookingCode', how = 'inner')

In [35]:
tt = u[['BookingCode', 'PassengerDocument_x', 'PassengerDocument_y', 'PassengerLastName_x', 'PassengerLastName_y',
        'PassengerFirstName_x', 'PassengerFirstName_y', 'PassengerSecondName_x', 'PassengerSecondName_y']]

In [38]:
pd.set_option('display.max_rows', len(tt))
    #print(df)
   # pd.reset_option('display.max_rows')

In [39]:
tt.query('PassengerDocument_x != PassengerDocument_y and \
    PassengerLastName_x != PassengerLastName_y and PassengerFirstName_x != PassengerFirstName_y\
         and PassengerSecondName_x != PassengerSecondName_y')

,BookingCode,PassengerDocument_x,PassengerDocument_y,PassengerLastName_x,PassengerLastName_y,PassengerFirstName_x,PassengerFirstName_y,PassengerSecondName_x,PassengerSecondName_y
173,AJUDMG,2095 686713,0526 304265,ELIZAROVA,MOROZOV,KAROLINA,RINAT,DEMIDOVNA,S.
174,AJUDMG,0526 304265,2095 686713,MOROZOV,ELIZAROVA,RINAT,KAROLINA,S.,DEMIDOVNA
205,ALZBPR,0653 510913,5883 111968,LOBANOV,SAVELEV,MIROSLAV,DENIS,EGOROVICH,M.
206,ALZBPR,5883 111968,0653 510913,SAVELEV,LOBANOV,DENIS,MIROSLAV,M.,EGOROVICH
309,AQSSTI,1655 512444,5975 713088,FADEEVA,LOSEVA,KSENIIA,ESENIIA,A.,I.
310,AQSSTI,5975 713088,1655 512444,LOSEVA,FADEEVA,ESENIIA,KSENIIA,I.,A.
549,BEXLEX,5596 558661,3229 154918,LAVROVA,BELOV,GALINA,GEORGII,RAMILEVNA,I.
550,BEXLEX,3229 154918,5596 558661,BELOV,LAVROVA,GEORGII,GALINA,I.,RAMILEVNA
617,BJPZIQ,2446 212515,09 2443852,DAVYDOVA,MIRONOVA,KARINA,IULIIA,M.,GRIGOREVNA
618,BJPZIQ,09 2443852,2446 212515,MIRONOVA,DAVYDOVA,IULIIA,KARINA,GRIGOREVNA,M.


In [41]:
u.query('BookingCode=="REYPYA"')

,ArrivalCity_x,BaggageState_x,BookingCode,CodeShare_x,DepartureDate_x,DepartureTime_x,FlightNumber_x,PassengerBirthDate_x,PassengerDocument_x,PassengerFirstName_x,PassengerLastName_x,PassengerSecondName_x,PassengerSex_x,TicketNumber_x,AgentInfo_x,ArrivalAirport_x,ArrivalDate_x,ArrivalTime_x,BaggageCount_x,DepartureAirport_x,Fare_x,Meal_x,PassengerAdditionalInfo_x,PassengerClass_x,ArrivalCountry_x,DepartureCity_x,DepartureCountry_x,PassengerNickName_x,ArrivalCity_y,BaggageState_y,CodeShare_y,DepartureDate_y,DepartureTime_y,FlightNumber_y,PassengerBirthDate_y,PassengerDocument_y,PassengerFirstName_y,PassengerLastName_y,PassengerSecondName_y,PassengerSex_y,TicketNumber_y,AgentInfo_y,ArrivalAirport_y,ArrivalDate_y,ArrivalTime_y,BaggageCount_y,DepartureAirport_y,Fare_y,Meal_y,PassengerAdditionalInfo_y,PassengerClass_y,ArrivalCountry_y,DepartureCity_y,DepartureCountry_y,PassengerNickName_y
9106,NaN,NaN,REYPYA,False,2017-01-18,14:30,SU1501,1988-06-23,18 9360460,TIHON,TIHONOV,TIHONOVICH,NaN,8.091321e+15,eDreams,SVO,2017-01-18,15:25,NaN,TJM,YFLXFT,NaN,9 FF#FB 944999141,Y,NaN,NaN,NaN,NaN,NaN,NaN,False,2017-01-18,14:30,SU1501,1988-06-23,18 9360460,TIHON,TIHONOV,TIHONOVICH,NaN,8.091321e+15,eDreams,SVO,2017-01-18,15:25,NaN,TJM,YFLXFT,NaN,9 FF#FB 944999141,Y,NaN,NaN,NaN,NaN
9107,NaN,NaN,REYPYA,False,2017-01-18,14:30,SU1501,1988-06-23,18 9360460,TIHON,TIHONOV,TIHONOVICH,NaN,8.091321e+15,eDreams,SVO,2017-01-18,15:25,NaN,TJM,YFLXFT,NaN,9 FF#FB 944999141,Y,NaN,NaN,NaN,NaN,Moscow,Transit,False,2017-01-18,14:30,SU1501,1988-06-23,6337 052814,TIKHON,TIKHONOV,T.,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9108,Moscow,Transit,REYPYA,False,2017-01-18,14:30,SU1501,1988-06-23,6337 052814,TIKHON,TIKHONOV,T.,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,2017-01-18,14:30,SU1501,1988-06-23,18 9360460,TIHON,TIHONOV,TIHONOVICH,NaN,8.091321e+15,eDreams,SVO,2017-01-18,15:25,NaN,TJM,YFLXFT,NaN,9 FF#FB 944999141,Y,NaN,NaN,NaN,NaN
9109,Moscow,Transit,REYPYA,False,2017-01-18,14:30,SU1501,1988-06-23,6337 052814,TIKHON,TIKHONOV,T.,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Moscow,Transit,False,2017-01-18,14:30,SU1501,1988-06-23,6337 052814,TIKHON,TIKHONOV,T.,Male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
8.091321e+15

8091321000000000.0

In [43]:
pd.reset_option('display.max_rows')

In [91]:
passes[passes['e-ticket']==8091321000000000]

,sequence,gender,name,agency,flight,from,to,airport_from,airport_to,date,time,pnr,e-ticket


In [172]:
passes = passes.rename(columns = {"e-ticket": 'TicketNumber', 'from': 'From'})

In [93]:
ps = passes.drop_duplicates()

In [320]:
oo  = passes.merge(joined, on = 'TicketNumber', how = 'inner')

In [132]:
oo['name'][4].split()#[0]

AttributeError: 'Series' object has no attribute 'split'

In [205]:
oo[['name', 'PassengerFirstName', 'PassengerLastName',  'PassengerSecondName', 'v11'
   , 'v12', 'v21', 'v22']]

,name,PassengerFirstName,PassengerLastName,PassengerSecondName,v11,v12,v21,v22
0,ISAKOVA VERA,VERA,ISAKOVA,DANIILOVNA,0.583333,1.000000,0.583333,1.000000
1,ZVYAGINTSEVA AL'BINA R,ALBINA,ZVIAGINTSEVA,ROMANOVNA,0.536585,0.878049,0.536585,0.878049
2,TAMARA TITOVA,TAMARA,TITOVA,E.,1.000000,0.461538,1.000000,0.461538
3,GALKIN FEDOR,FEDOR,GALKIN,ADELEVICH,0.500000,1.000000,0.500000,1.000000
4,NATAL'YA K KALACHEVA,NATALIA,KALACHEVA,KIRILLOVNA,0.864865,0.486486,0.864865,0.486486
...,...,...,...,...,...,...,...,...
58951,TITOVA VLADISLAVA,VLADISLAVA,TITOVA,BOGDANOVNA,0.588235,1.000000,0.588235,1.000000
58952,VERA KOLPAKOVA,VERA,KOLPAKOVA,K.,1.000000,0.642857,1.000000,0.642857
58953,MOROZOV MIROSLAV,MIROSLAV,MOROZOV,DEMIDOVICH,0.500000,1.000000,0.500000,1.000000
58954,KOCHERGINA KIRA,KIRA,KOCHERGINA,ROBERTOVNA,0.666667,1.000000,0.666667,1.000000


In [177]:
oo.query('(gender == "MR" & PassengerSex=="Female") | (gender == "MRS" & PassengerSex=="Male")')

,sequence,gender,name,agency,flight,From,to,airport_from,airport_to,date,time,pnr,TicketNumber,class,ArrivalCity,BaggageState,BookingCode,CodeShare,DepartureDate,DepartureTime,FlightNumber,PassengerBirthDate,PassengerDocument,PassengerFirstName,PassengerLastName,PassengerSecondName,PassengerSex,AgentInfo,ArrivalAirport,ArrivalDate,ArrivalTime,BaggageCount,DepartureAirport,Fare,Meal,PassengerAdditionalInfo,PassengerClass,ArrivalCountry,DepartureCity,DepartureCountry,PassengerNickName


In [178]:
oo.query("flight != FlightNumber")

,sequence,gender,name,agency,flight,From,to,airport_from,airport_to,date,time,pnr,TicketNumber,class,ArrivalCity,BaggageState,BookingCode,CodeShare,DepartureDate,DepartureTime,FlightNumber,PassengerBirthDate,PassengerDocument,PassengerFirstName,PassengerLastName,PassengerSecondName,PassengerSex,AgentInfo,ArrivalAirport,ArrivalDate,ArrivalTime,BaggageCount,DepartureAirport,Fare,Meal,PassengerAdditionalInfo,PassengerClass,ArrivalCountry,DepartureCity,DepartureCountry,PassengerNickName


In [181]:
oo.query("From.str.upper() != DepartureCity.str.upper() & DepartureCity==DepartureCity | \
         to.str.upper() != ArrivalCity.str.upper() & ArrivalCity==ArrivalCity")

,sequence,gender,name,agency,flight,From,to,airport_from,airport_to,date,time,pnr,TicketNumber,class,ArrivalCity,BaggageState,BookingCode,CodeShare,DepartureDate,DepartureTime,FlightNumber,PassengerBirthDate,PassengerDocument,PassengerFirstName,PassengerLastName,PassengerSecondName,PassengerSex,AgentInfo,ArrivalAirport,ArrivalDate,ArrivalTime,BaggageCount,DepartureAirport,Fare,Meal,PassengerAdditionalInfo,PassengerClass,ArrivalCountry,DepartureCity,DepartureCountry,PassengerNickName


In [182]:
oo.query( "airport_from.str.upper() != DepartureAirport.str.upper() & DepartureAirport==DepartureAirport | \
         airport_to.str.upper() != ArrivalAirport.str.upper() & ArrivalAirport==ArrivalAirport")

,sequence,gender,name,agency,flight,From,to,airport_from,airport_to,date,time,pnr,TicketNumber,class,ArrivalCity,BaggageState,BookingCode,CodeShare,DepartureDate,DepartureTime,FlightNumber,PassengerBirthDate,PassengerDocument,PassengerFirstName,PassengerLastName,PassengerSecondName,PassengerSex,AgentInfo,ArrivalAirport,ArrivalDate,ArrivalTime,BaggageCount,DepartureAirport,Fare,Meal,PassengerAdditionalInfo,PassengerClass,ArrivalCountry,DepartureCity,DepartureCountry,PassengerNickName


In [183]:
oo.query( "date != DepartureDate  & DepartureDate==DepartureDate | \
         time != DepartureTime & DepartureTime==DepartureTime")

,sequence,gender,name,agency,flight,From,to,airport_from,airport_to,date,time,pnr,TicketNumber,class,ArrivalCity,BaggageState,BookingCode,CodeShare,DepartureDate,DepartureTime,FlightNumber,PassengerBirthDate,PassengerDocument,PassengerFirstName,PassengerLastName,PassengerSecondName,PassengerSex,AgentInfo,ArrivalAirport,ArrivalDate,ArrivalTime,BaggageCount,DepartureAirport,Fare,Meal,PassengerAdditionalInfo,PassengerClass,ArrivalCountry,DepartureCity,DepartureCountry,PassengerNickName


In [191]:
oo.query( "pnr != BookingCode & BookingCode==BookingCode")

,sequence,gender,name,agency,flight,From,to,airport_from,airport_to,date,time,pnr,TicketNumber,Class,ArrivalCity,BaggageState,BookingCode,CodeShare,DepartureDate,DepartureTime,FlightNumber,PassengerBirthDate,PassengerDocument,PassengerFirstName,PassengerLastName,PassengerSecondName,PassengerSex,AgentInfo,ArrivalAirport,ArrivalDate,ArrivalTime,BaggageCount,DepartureAirport,Fare,Meal,PassengerAdditionalInfo,PassengerClass,ArrivalCountry,DepartureCity,DepartureCountry,PassengerNickName
842,27,MR,ILDAR NIKULIN,NaN,SU1249,ORENBURG,MOSCOW,REN,SVO,2017-01-01,19:50,UHDTJA,8176809349147027,Y,Moscow,Registered,CWPEID,False,2017-01-01,19:50,SU1249,1991-12-24,5670 353504,ILDAR,NIKULIN,D.,Male,Aerobilet,SVO,2017-01-01,20:15,1PC,REN,YRSTYW,VLML,NaN,Y,NaN,NaN,NaN,NaN


In [188]:
oo = oo.rename(columns={'class': 'Class'})

In [192]:
oo.query("Class != PassengerClass & PassengerClass==PassengerClass")

,sequence,gender,name,agency,flight,From,to,airport_from,airport_to,date,time,pnr,TicketNumber,Class,ArrivalCity,BaggageState,BookingCode,CodeShare,DepartureDate,DepartureTime,FlightNumber,PassengerBirthDate,PassengerDocument,PassengerFirstName,PassengerLastName,PassengerSecondName,PassengerSex,AgentInfo,ArrivalAirport,ArrivalDate,ArrivalTime,BaggageCount,DepartureAirport,Fare,Meal,PassengerAdditionalInfo,PassengerClass,ArrivalCountry,DepartureCity,DepartureCountry,PassengerNickName


In [209]:
oo.query('v11 < 0.7 & v12 < 0.7 & v21 < 0.7 & v22 < 0.7')[['name', 'PassengerFirstName', 'PassengerLastName',  'PassengerSecondName', 'v11'
   , 'v12', 'v21', 'v22']]

,name,PassengerFirstName,PassengerLastName,PassengerSecondName,v11,v12,v21,v22
39750,YUDIN IL'YA S,ILIA,IUDIN,SAMIROVICH,0.347826,0.695652,0.347826,0.695652
56595,D'YAKOV YURIY V,IURII,DIAKOV,VIKTOROVICH,0.370370,0.666667,0.370370,0.666667


In [ ]:
import difflib as dfl

In [204]:
import pandas as pd
import difflib as dfl
#df = pd.DataFrame({'A': ["john doe", " john doe", 'John'], 'B': [' john doe', 'eddie murphy', 'batman']})
#df['VAR1'] = 
oo['v11'] = oo.apply(lambda x : dfl.SequenceMatcher(None, x['name'], x['PassengerFirstName'] + ' ' + x['PassengerLastName']).ratio(),axis=1)
oo['v12'] = oo.apply(lambda x : dfl.SequenceMatcher(None, x['name'], x['PassengerLastName'] + ' ' + x['PassengerFirstName']).ratio(),axis=1)
oo['v21'] = oo.apply(lambda x : dfl.SequenceMatcher(None, x['PassengerFirstName'] + ' ' + x['PassengerLastName'], x['name']).ratio(),axis=1)
oo['v22'] = oo.apply(lambda x : dfl.SequenceMatcher(None, x['PassengerLastName'] + ' ' + x['PassengerFirstName'], x['name']).ratio(),axis=1)

In [213]:
joined.isna().sum()

ArrivalCity                 72815
BaggageState                84565
BookingCode                 42085
CodeShare                       0
DepartureDate                   0
DepartureTime                   0
FlightNumber                    0
PassengerBirthDate          42398
PassengerDocument               0
PassengerFirstName              0
PassengerLastName               0
PassengerSecondName             0
PassengerSex                72815
TicketNumber                41991
AgentInfo                   95898
ArrivalAirport              69639
ArrivalDate                 83955
ArrivalTime                 83955
BaggageCount               154649
DepartureAirport            69639
Fare                        83955
Meal                       181567
PassengerAdditionalInfo    109183
PassengerClass              83955
ArrivalCountry             201950
DepartureCity              201950
DepartureCountry           201950
PassengerNickName          201950
dtype: int64

In [212]:
joined

,ArrivalCity,BaggageState,BookingCode,CodeShare,DepartureDate,DepartureTime,FlightNumber,PassengerBirthDate,PassengerDocument,PassengerFirstName,PassengerLastName,PassengerSecondName,PassengerSex,TicketNumber,AgentInfo,ArrivalAirport,ArrivalDate,ArrivalTime,BaggageCount,DepartureAirport,Fare,Meal,PassengerAdditionalInfo,PassengerClass,ArrivalCountry,DepartureCity,DepartureCountry,PassengerNickName
0,Moscow,Transit,WMXPVY,False,2017-01-01,09:15,SU1001,1999-12-13,5140 683757,ALISA,ABRAMOVA,R.,Female,4.178367e+15,Travelgenio,SVO,2017-01-01,12:10,1PC,KGD,YFLXYW,RVML,F,Y,NaN,NaN,NaN,NaN
1,NaN,NaN,ULCPQH,False,2017-01-01,09:15,SU1001,NaT,7477 017451,KRISTINA,ARTEMOVA,MARATOVNA,NaN,7.935022e+14,OneTwoTrip,SVO,2017-01-01,12:10,NaN,KGD,YFLXEY,NaN,NaN,Y,NaN,NaN,NaN,NaN
2,NaN,NaN,YVICRN,False,2017-01-01,09:15,SU1001,NaT,6840 635650,ALEKSANDR,BABUSHKIN,ALEKSANDROVICH,NaN,2.360894e+15,eDreams,SVO,2017-01-01,12:10,1PC,KGD,YSTNUB,NaN,4 FF#FB 483772175,Y,NaN,NaN,NaN,NaN
3,Moscow,Registered,NaN,False,2017-01-01,09:15,SU1001,1974-07-09,1205 586135,MARK,BEREZIN,A.,Male,8.200536e+15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,DBEVCU,False,2017-01-01,09:15,SU1001,1990-05-04,6601 454145,VASILISA,BLINOVA,ILDAROVNA,NaN,7.315197e+15,City.Travel,SVO,2017-01-01,12:10,1PC,KGD,YGRPPY,VGML,0 FF#SU 806242829,Y,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239712,NaN,NaN,VERHMP,False,2018-01-01,09:55,SU6183,NaT,2941 115924,EGOR,LEVIN,SAVELEVICH,NaN,2.038332e+15,City.Travel,ROV,2018-01-01,11:55,NaN,SVO,YSTNII,MOML,#,Y,NaN,NaN,NaN,NaN
239713,Moscow,None,DWPNNZ,False,2018-01-01,12:45,SU6184,1978-03-30,2191 291628,ANZHELIKA,BLINOVA,MAKAREVNA,Female,6.149777e+15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
239714,Moscow,Registered,RWUBBA,False,2018-01-01,16:30,SU6286,1973-05-01,2022 416117,ALEKSANDRA,KORCHAGINA,GLEBOVNA,Female,1.650099e+15,NaN,SVO,2018-01-01,16:05,NaN,PKC,YFLXGK,STML,K FF#KE 978286566,Y,NaN,NaN,NaN,NaN
239715,Khabarovsk,Registered,STCVTK,False,2018-01-01,17:40,SU6287,1995-04-28,6815 172060,SVETLANA,ZLOBINA,D.,Female,2.360753e+15,Aerobilet,KHV,2018-01-02,08:15,NaN,SVO,YRSTQE,NaN,U FF#SU 316501612,Y,NaN,NaN,NaN,NaN


In [219]:
oo.replace(['MRS', 'MR'], ['Female', 'Male'], inplace = True)

In [216]:
joined[joined['TicketNumber']==oo['TicketNumber']]

ValueError: Can only compare identically-labeled Series objects

In [221]:
joined

,ArrivalCity,BaggageState,BookingCode,CodeShare,DepartureDate,DepartureTime,FlightNumber,PassengerBirthDate,PassengerDocument,PassengerFirstName,PassengerLastName,PassengerSecondName,PassengerSex,TicketNumber,AgentInfo,ArrivalAirport,ArrivalDate,ArrivalTime,BaggageCount,DepartureAirport,Fare,Meal,PassengerAdditionalInfo,PassengerClass,ArrivalCountry,DepartureCity,DepartureCountry,PassengerNickName
0,Moscow,Transit,WMXPVY,False,2017-01-01,09:15,SU1001,1999-12-13,5140 683757,ALISA,ABRAMOVA,R.,Female,4.178367e+15,Travelgenio,SVO,2017-01-01,12:10,1PC,KGD,YFLXYW,RVML,F,Y,NaN,NaN,NaN,NaN
1,NaN,NaN,ULCPQH,False,2017-01-01,09:15,SU1001,NaT,7477 017451,KRISTINA,ARTEMOVA,MARATOVNA,NaN,7.935022e+14,OneTwoTrip,SVO,2017-01-01,12:10,NaN,KGD,YFLXEY,NaN,NaN,Y,NaN,NaN,NaN,NaN
2,NaN,NaN,YVICRN,False,2017-01-01,09:15,SU1001,NaT,6840 635650,ALEKSANDR,BABUSHKIN,ALEKSANDROVICH,NaN,2.360894e+15,eDreams,SVO,2017-01-01,12:10,1PC,KGD,YSTNUB,NaN,4 FF#FB 483772175,Y,NaN,NaN,NaN,NaN
3,Moscow,Registered,NaN,False,2017-01-01,09:15,SU1001,1974-07-09,1205 586135,MARK,BEREZIN,A.,Male,8.200536e+15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,DBEVCU,False,2017-01-01,09:15,SU1001,1990-05-04,6601 454145,VASILISA,BLINOVA,ILDAROVNA,NaN,7.315197e+15,City.Travel,SVO,2017-01-01,12:10,1PC,KGD,YGRPPY,VGML,0 FF#SU 806242829,Y,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239712,NaN,NaN,VERHMP,False,2018-01-01,09:55,SU6183,NaT,2941 115924,EGOR,LEVIN,SAVELEVICH,NaN,2.038332e+15,City.Travel,ROV,2018-01-01,11:55,NaN,SVO,YSTNII,MOML,#,Y,NaN,NaN,NaN,NaN
239713,Moscow,None,DWPNNZ,False,2018-01-01,12:45,SU6184,1978-03-30,2191 291628,ANZHELIKA,BLINOVA,MAKAREVNA,Female,6.149777e+15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
239714,Moscow,Registered,RWUBBA,False,2018-01-01,16:30,SU6286,1973-05-01,2022 416117,ALEKSANDRA,KORCHAGINA,GLEBOVNA,Female,1.650099e+15,NaN,SVO,2018-01-01,16:05,NaN,PKC,YFLXGK,STML,K FF#KE 978286566,Y,NaN,NaN,NaN,NaN
239715,Khabarovsk,Registered,STCVTK,False,2018-01-01,17:40,SU6287,1995-04-28,6815 172060,SVETLANA,ZLOBINA,D.,Female,2.360753e+15,Aerobilet,KHV,2018-01-02,08:15,NaN,SVO,YRSTQE,NaN,U FF#SU 316501612,Y,NaN,NaN,NaN,NaN


In [231]:
oo = oo.rename(columns=dict([c, c + '_2'] for c in oo.columns))

In [322]:
joined2 = joined.merge(oo,  left_on = 'TicketNumber', right_on = 'TicketNumber', how = 'left')

In [237]:
joined2[joined2['ArrivalCity'] != joined2['ArrivalCity']]['ArrivalCity'] == joined2['to_2']

ValueError: Can only compare identically-labeled Series objects

In [239]:
joined2.where(joined2['ArrivalCity'] != joined2['ArrivalCity'], joined2['ArrivalCity'] == joined2['to_2'])

#joined2[joined2['ArrivalCity'] != joined2['ArrivalCity']]['ArrivalCity'] == joined2['to_2']

ValueError: Must specify axis=0 or 1

In [323]:
joined2

,ArrivalCity_x,BaggageState_x,BookingCode_x,CodeShare_x,DepartureDate_x,DepartureTime_x,FlightNumber_x,PassengerBirthDate_x,PassengerDocument_x,PassengerFirstName_x,PassengerLastName_x,PassengerSecondName_x,PassengerSex_x,TicketNumber,AgentInfo_x,ArrivalAirport_x,ArrivalDate_x,ArrivalTime_x,BaggageCount_x,DepartureAirport_x,Fare_x,Meal_x,PassengerAdditionalInfo_x,PassengerClass_x,ArrivalCountry_x,DepartureCity_x,DepartureCountry_x,PassengerNickName_x,sequence,gender,name,agency,flight,From,to,airport_from,airport_to,date,time,pnr,class,ArrivalCity_y,BaggageState_y,BookingCode_y,CodeShare_y,DepartureDate_y,DepartureTime_y,FlightNumber_y,PassengerBirthDate_y,PassengerDocument_y,PassengerFirstName_y,PassengerLastName_y,PassengerSecondName_y,PassengerSex_y,AgentInfo_y,ArrivalAirport_y,ArrivalDate_y,ArrivalTime_y,BaggageCount_y,DepartureAirport_y,Fare_y,Meal_y,PassengerAdditionalInfo_y,PassengerClass_y,ArrivalCountry_y,DepartureCity_y,DepartureCountry_y,PassengerNickName_y
0,Moscow,Transit,WMXPVY,False,2017-01-01,09:15,SU1001,1999-12-13,5140 683757,ALISA,ABRAMOVA,R.,Female,4.178367e+15,Travelgenio,SVO,2017-01-01,12:10,1PC,KGD,YFLXYW,RVML,F,Y,NaN,NaN,NaN,NaN,77.0,MRS,ALISA ABRAMOVA,NaN,SU1001,KALININGRAD,MOSCOW,KGD,SVO,2017-01-01,09:15,WMXPVY,Y,Moscow,Transit,WMXPVY,False,2017-01-01,09:15,SU1001,1999-12-13,5140 683757,ALISA,ABRAMOVA,R.,Female,Travelgenio,SVO,2017-01-01,12:10,1PC,KGD,YFLXYW,RVML,F,Y,NaN,NaN,NaN,NaN
1,NaN,NaN,ULCPQH,False,2017-01-01,09:15,SU1001,NaT,7477 017451,KRISTINA,ARTEMOVA,MARATOVNA,NaN,7.935022e+14,OneTwoTrip,SVO,2017-01-01,12:10,NaN,KGD,YFLXEY,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,YVICRN,False,2017-01-01,09:15,SU1001,NaT,6840 635650,ALEKSANDR,BABUSHKIN,ALEKSANDROVICH,NaN,2.360894e+15,eDreams,SVO,2017-01-01,12:10,1PC,KGD,YSTNUB,NaN,4 FF#FB 483772175,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Moscow,Registered,NaN,False,2017-01-01,09:15,SU1001,1974-07-09,1205 586135,MARK,BEREZIN,A.,Male,8.200536e+15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,DBEVCU,False,2017-01-01,09:15,SU1001,1990-05-04,6601 454145,VASILISA,BLINOVA,ILDAROVNA,NaN,7.315197e+15,City.Travel,SVO,2017-01-01,12:10,1PC,KGD,YGRPPY,VGML,0 FF#SU 806242829,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241756,NaN,NaN,VERHMP,False,2018-01-01,09:55,SU6183,NaT,2941 115924,EGOR,LEVIN,SAVELEVICH,NaN,2.038332e+15,City.Travel,ROV,2018-01-01,11:55,NaN,SVO,YSTNII,MOML,#,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
241757,Moscow,None,DWPNNZ,False,2018-01-01,12:45,SU6184,1978-03-30,2191 291628,ANZHELIKA,BLINOVA,MAKAREVNA,Female,6.149777e+15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
241758,Moscow,Registered,RWUBBA,False,2018-01-01,16:30,SU6286,1973-05-01,2022 416117,ALEKSANDRA,KORCHAGINA,GLEBOVNA,Female,1.650099e+15,NaN,SVO,2018-01-01,16:05,NaN,PKC,YFLXGK,STML,K FF#KE 978286566,Y,NaN,NaN,NaN,NaN,Na

In [249]:
oo

,sequence_2,gender_2,name_2,agency_2,flight_2,From_2,to_2,airport_from_2,airport_to_2,date_2,time_2,pnr_2,TicketNumber_2,Class_2,ArrivalCity_2,BaggageState_2,BookingCode_2,CodeShare_2,DepartureDate_2,DepartureTime_2,FlightNumber_2,PassengerBirthDate_2,PassengerDocument_2,PassengerFirstName_2,PassengerLastName_2,PassengerSecondName_2,PassengerSex_2,AgentInfo_2,ArrivalAirport_2,ArrivalDate_2,ArrivalTime_2,BaggageCount_2,DepartureAirport_2,Fare_2,Meal_2,PassengerAdditionalInfo_2,PassengerClass_2,ArrivalCountry_2,DepartureCity_2,DepartureCountry_2,PassengerNickName_2,name2_2,v11_2,v12_2,v21_2,v22_2
0,77,Female,ISAKOVA VERA,NaN,SU5642,KHABAROVSK,NOVOSIBIRSK,KHV,OVB,2017-01-01,19:25,PCJDQU,2207650722148272,Y,NaN,NaN,PCJDQU,False,2017-01-01,19:25,SU5642,NaT,37 5110651,VERA,ISAKOVA,DANIILOVNA,NaN,Aeroflot,OVB,2017-01-01,21:45,0PC,KHV,YFLXZY,VGML,NaN,Y,NaN,NaN,NaN,NaN,VERA ISAKOVA,0.583333,1.000000,0.583333,1.000000
1,81,Female,ZVYAGINTSEVA AL'BINA R,NaN,SU5642,KHABAROVSK,NOVOSIBIRSK,KHV,OVB,2017-01-01,19:25,UDXDFD,5088998931607787,Y,Novosibirsk,Registered,UDXDFD,False,2017-01-01,19:25,SU5642,1972-07-08,1481 672135,ALBINA,ZVIAGINTSEVA,ROMANOVNA,Female,trip.ru,OVB,2017-01-01,21:45,NaN,KHV,YFLXEV,NaN,F,Y,NaN,NaN,NaN,NaN,ALBINA ZVIAGINTSEVA,0.536585,0.878049,0.536585,0.878049
2,51,Female,TAMARA TITOVA,NaN,SU1445,IRKUTSK,MOSCOW,IKT,SVO,2017-01-01,19:40,FQSXBE,9809048308727614,Y,Moscow,Delayed,NaN,False,2017-01-01,19:40,SU1445,1995-08-03,9383 317316,TAMARA,TITOVA,E.,Female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TAMARA TITOVA,1.000000,0.461538,1.000000,0.461538
3,87,Male,GALKIN FEDOR,FB 372130571,SU1445,IRKUTSK,MOSCOW,IKT,SVO,2017-01-01,19:40,QDWGSP,3990603578632925,Y,Moscow,NaN,QDWGSP,False,2017-01-01,19:40,SU1445,NaT,8011 940172,FEDOR,GALKIN,ADELEVICH,Male,Kiwi.com,SVO,2017-01-01,20:50,NaN,IKT,YGRPOA,HNML,FF#FB 372130571,Y,Russian Federation,Irkutsk,Russian Federation,FrequentFlyer86569,FEDOR GALKIN,0.500000,1.000000,0.500000,1.000000
4,14,Female,NATAL'YA K KALACHEVA,NaN,SU1445,IRKUTSK,MOSCOW,IKT,SVO,2017-01-01,19:40,RKHWUW,9050068989666768,Y,Moscow,Registered,RKHWUW,False,2017-01-01,19:40,SU1445,1979-01-09,6871 143218,NATALIA,KALACHEVA,KIRILLOVNA,Female,trip.ru,SVO,2017-01-01,20:50,NaN,IKT,YRSTAC,VGML,F,Y,NaN,NaN,NaN,NaN,NATALIA KALACHEVA,0.864865,0.486486,0.864865,0.486486
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58951,32,Female,TITOVA VLADISLAVA,NaN,SU1200,MOSCOW,PERM,SVO,PEE,2018-01-01,06:55,LPEXSV,7292480289373779,Y,Perm,Delayed,LPEXSV,False,2018-01-01,06:55,SU1200,1970-05-28,3047 735668,VLADISLAVA,TITOVA,BOGDANOVNA,Female,trip.ru,PEE,2018-01-01,11:05,0PC,SVO,YRSTCF,ORML,#,Y,NaN,NaN,NaN,NaN,VLADISLAVA TITOVA,0.588235,1.000000,0.588235,1.000000
58952,69,Female,VERA KOLPAKOVA,NaN,SU4602,KOMSOMOLSK,VLADIVOSTOK,KXK,VVO,2018-01-01,13:10,ERICQE,830710434038463,Y,Vladivostok,None,ERICQE,True,2018-01-01,13:10,SU4602,1983-04-22,0185 106233,VERA,KOLPAKOVA,K.,Female,OneTwoTrip,VVO,2018-01-01,15:10,2PC,KXK,YRSTPW,NaN,NaN,Y,NaN,NaN,NaN,NaN,VERA KOLPAKOVA,1.000000,0.642857,1.000000,0.642857
58953,10,Male,MOROZOV MIROSLAV,NaN,SU1280,MOSCOW,KAZAN,SVO,KZN,2018-01-01,10:55,JNWDUC,6827266375620492,A,Kazan,Registered,JNWDUC,False,2018-01-01,10:55,SU1280,1975-08-02,8823 858714,MIROSLAV,MOROZOV,DEMIDOVICH,Male,NaN,KZN,NaN,NaN,NaN,SVO,NaN,NaN,NaN,NaN,Russian Federation,Moscow,Russian Federation,FrequentFlyer39532,MIROSLAV MOROZOV,0.500000,1.000000,0.500000,1.000000
58954,69,Female,KOCHERGINA KIRA,NaN,SU6171,MOSCOW,ORENBURG,SVO,REN,2018-01-01,17:15,IYJVXN,3126950449728908,Y,Orenburg,None,IYJVXN,False,2018-01-01,17:15,SU6171,1970-06-25,2563 521202,KIRA,KOCHERGINA,ROBERTOVNA,Female,City.Travel,REN,2018-01-01,21:30,2PC,SVO,YRSTAF,NaN,F,Y,Russian Federation,Moscow,Russian Federation,FrequentFlyer45921,KIRA KOCHERGINA,0.666667,1.000000,0.666667,1.000000


In [254]:
joined2

,ArrivalCity,BaggageState,BookingCode,CodeShare,DepartureDate,DepartureTime,FlightNumber,PassengerBirthDate,PassengerDocument,PassengerFirstName,PassengerLastName,PassengerSecondName,PassengerSex,TicketNumber,AgentInfo,ArrivalAirport,ArrivalDate,ArrivalTime,BaggageCount,DepartureAirport,Fare,Meal,PassengerAdditionalInfo,PassengerClass,ArrivalCountry,DepartureCity,DepartureCountry,PassengerNickName,sequence_2,gender_2,name_2,agency_2,flight_2,From_2,to_2,airport_from_2,airport_to_2,date_2,time_2,pnr_2,TicketNumber_2,Class_2,ArrivalCity_2,BaggageState_2,BookingCode_2,CodeShare_2,DepartureDate_2,DepartureTime_2,FlightNumber_2,PassengerBirthDate_2,PassengerDocument_2,PassengerFirstName_2,PassengerLastName_2,PassengerSecondName_2,PassengerSex_2,AgentInfo_2,ArrivalAirport_2,ArrivalDate_2,ArrivalTime_2,BaggageCount_2,DepartureAirport_2,Fare_2,Meal_2,PassengerAdditionalInfo_2,PassengerClass_2,ArrivalCountry_2,DepartureCity_2,DepartureCountry_2,PassengerNickName_2
0,Moscow,Transit,WMXPVY,False,2017-01-01,09:15,SU1001,1999-12-13,5140 683757,ALISA,ABRAMOVA,R.,Female,4.178367e+15,Travelgenio,SVO,2017-01-01,12:10,1PC,KGD,YFLXYW,RVML,F,Y,NaN,NaN,NaN,NaN,77.0,Female,ALISA ABRAMOVA,NaN,SU1001,KALININGRAD,MOSCOW,KGD,SVO,2017-01-01,09:15,WMXPVY,4.178367e+15,Y,Moscow,Transit,WMXPVY,False,2017-01-01,09:15,SU1001,1999-12-13,5140 683757,ALISA,ABRAMOVA,R.,Female,Travelgenio,SVO,2017-01-01,12:10,1PC,KGD,YFLXYW,RVML,F,Y,NaN,NaN,NaN,NaN
1,NaN,NaN,ULCPQH,False,2017-01-01,09:15,SU1001,NaT,7477 017451,KRISTINA,ARTEMOVA,MARATOVNA,NaN,7.935022e+14,OneTwoTrip,SVO,2017-01-01,12:10,NaN,KGD,YFLXEY,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,YVICRN,False,2017-01-01,09:15,SU1001,NaT,6840 635650,ALEKSANDR,BABUSHKIN,ALEKSANDROVICH,NaN,2.360894e+15,eDreams,SVO,2017-01-01,12:10,1PC,KGD,YSTNUB,NaN,4 FF#FB 483772175,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Moscow,Registered,NaN,False,2017-01-01,09:15,SU1001,1974-07-09,1205 586135,MARK,BEREZIN,A.,Male,8.200536e+15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,DBEVCU,False,2017-01-01,09:15,SU1001,1990-05-04,6601 454145,VASILISA,BLINOVA,ILDAROVNA,NaN,7.315197e+15,City.Travel,SVO,2017-01-01,12:10,1PC,KGD,YGRPPY,VGML,0 FF#SU 806242829,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241756,NaN,NaN,VERHMP,False,2018-01-01,09:55,SU6183,NaT,2941 115924,EGOR,LEVIN,SAVELEVICH,NaN,2.038332e+15,City.Travel,ROV,2018-01-01,11:55,NaN,SVO,YSTNII,MOML,#,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
241757,Moscow,None,DWPNNZ,False,2018-01-01,12:45,SU6184,1978-03-30,2191 291628,ANZHELIKA,BLINOVA,MAKAREVNA,Female,6.149777e+15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
241758,Moscow,Registered,RWUBBA,False,2018-01-01,16:30,SU6286,1973-05-01,2022 416117,ALEKSANDRA,KORCHAGINA,GLEBOVNA,Female,1.650099e+15,NaN,SVO,2018-01-01,16:05,NaN,PKC,YFLXGK,STML,K FF#KE

In [259]:
joined2[['ArrivalCity', 'to_2']].max()

Series([], dtype: float64)

In [268]:
a = joined2['ArrivalCity'].where(joined2['ArrivalCity'] !=joined2['ArrivalCity'], joined2['to_2'])

In [267]:
joined2

,ArrivalCity,BaggageState,BookingCode,CodeShare,DepartureDate,DepartureTime,FlightNumber,PassengerBirthDate,PassengerDocument,PassengerFirstName,PassengerLastName,PassengerSecondName,PassengerSex,TicketNumber,AgentInfo,ArrivalAirport,ArrivalDate,ArrivalTime,BaggageCount,DepartureAirport,Fare,Meal,PassengerAdditionalInfo,PassengerClass,ArrivalCountry,DepartureCity,DepartureCountry,PassengerNickName,sequence_2,gender_2,name_2,agency_2,flight_2,From_2,to_2,airport_from_2,airport_to_2,date_2,time_2,pnr_2,TicketNumber_2,Class_2,ArrivalCity_2,BaggageState_2,BookingCode_2,CodeShare_2,DepartureDate_2,DepartureTime_2,FlightNumber_2,PassengerBirthDate_2,PassengerDocument_2,PassengerFirstName_2,PassengerLastName_2,PassengerSecondName_2,PassengerSex_2,AgentInfo_2,ArrivalAirport_2,ArrivalDate_2,ArrivalTime_2,BaggageCount_2,DepartureAirport_2,Fare_2,Meal_2,PassengerAdditionalInfo_2,PassengerClass_2,ArrivalCountry_2,DepartureCity_2,DepartureCountry_2,PassengerNickName_2
0,Moscow,Transit,WMXPVY,False,2017-01-01,09:15,SU1001,1999-12-13,5140 683757,ALISA,ABRAMOVA,R.,Female,4.178367e+15,Travelgenio,SVO,2017-01-01,12:10,1PC,KGD,YFLXYW,RVML,F,Y,NaN,NaN,NaN,NaN,77.0,Female,ALISA ABRAMOVA,NaN,SU1001,KALININGRAD,MOSCOW,KGD,SVO,2017-01-01,09:15,WMXPVY,4.178367e+15,Y,Moscow,Transit,WMXPVY,False,2017-01-01,09:15,SU1001,1999-12-13,5140 683757,ALISA,ABRAMOVA,R.,Female,Travelgenio,SVO,2017-01-01,12:10,1PC,KGD,YFLXYW,RVML,F,Y,NaN,NaN,NaN,NaN
1,NaN,NaN,ULCPQH,False,2017-01-01,09:15,SU1001,NaT,7477 017451,KRISTINA,ARTEMOVA,MARATOVNA,NaN,7.935022e+14,OneTwoTrip,SVO,2017-01-01,12:10,NaN,KGD,YFLXEY,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,YVICRN,False,2017-01-01,09:15,SU1001,NaT,6840 635650,ALEKSANDR,BABUSHKIN,ALEKSANDROVICH,NaN,2.360894e+15,eDreams,SVO,2017-01-01,12:10,1PC,KGD,YSTNUB,NaN,4 FF#FB 483772175,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Moscow,Registered,NaN,False,2017-01-01,09:15,SU1001,1974-07-09,1205 586135,MARK,BEREZIN,A.,Male,8.200536e+15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,DBEVCU,False,2017-01-01,09:15,SU1001,1990-05-04,6601 454145,VASILISA,BLINOVA,ILDAROVNA,NaN,7.315197e+15,City.Travel,SVO,2017-01-01,12:10,1PC,KGD,YGRPPY,VGML,0 FF#SU 806242829,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241756,NaN,NaN,VERHMP,False,2018-01-01,09:55,SU6183,NaT,2941 115924,EGOR,LEVIN,SAVELEVICH,NaN,2.038332e+15,City.Travel,ROV,2018-01-01,11:55,NaN,SVO,YSTNII,MOML,#,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
241757,Moscow,None,DWPNNZ,False,2018-01-01,12:45,SU6184,1978-03-30,2191 291628,ANZHELIKA,BLINOVA,MAKAREVNA,Female,6.149777e+15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
241758,Moscow,Registered,RWUBBA,False,2018-01-01,16:30,SU6286,1973-05-01,2022 416117,ALEKSANDRA,KORCHAGINA,GLEBOVNA,Female,1.650099e+15,NaN,SVO,2018-01-01,16:05,NaN,PKC,YFLXGK,STML,K FF#KE

In [270]:
a.notna().sum()

38282

In [293]:
joined2['ArrivalCity'].notna().sum()

190643

In [292]:
print(datetime.now())
def my_fun(x, var1, var2):
    if x[var1] == x[var1] :
        return x[var1]
    else:
        return x[var2]
       


print(datetime.now())

2021-11-19 02:20:46.702710
2021-11-19 02:20:51.237514


In [ ]:
def my_fun (var1,var2,var3):
    df[var3]= np.where((df[var1]-df[var2])>0, df[var1]-df[var2], 0)
    return df

In [291]:
joined2

,ArrivalCity,BaggageState,BookingCode,CodeShare,DepartureDate,DepartureTime,FlightNumber,PassengerBirthDate,PassengerDocument,PassengerFirstName,PassengerLastName,PassengerSecondName,PassengerSex,TicketNumber,AgentInfo,ArrivalAirport,ArrivalDate,ArrivalTime,BaggageCount,DepartureAirport,Fare,Meal,PassengerAdditionalInfo,PassengerClass,ArrivalCountry,DepartureCity,DepartureCountry,PassengerNickName,sequence_2,gender_2,name_2,agency_2,flight_2,From_2,to_2,airport_from_2,airport_to_2,date_2,time_2,pnr_2,TicketNumber_2,Class_2,ArrivalCity_2,BaggageState_2,BookingCode_2,CodeShare_2,DepartureDate_2,DepartureTime_2,FlightNumber_2,PassengerBirthDate_2,PassengerDocument_2,PassengerFirstName_2,PassengerLastName_2,PassengerSecondName_2,PassengerSex_2,AgentInfo_2,ArrivalAirport_2,ArrivalDate_2,ArrivalTime_2,BaggageCount_2,DepartureAirport_2,Fare_2,Meal_2,PassengerAdditionalInfo_2,PassengerClass_2,ArrivalCountry_2,DepartureCity_2,DepartureCountry_2,PassengerNickName_2,arr22
0,Moscow,Transit,WMXPVY,False,2017-01-01,09:15,SU1001,1999-12-13,5140 683757,ALISA,ABRAMOVA,R.,Female,4.178367e+15,Travelgenio,SVO,2017-01-01,12:10,1PC,KGD,YFLXYW,RVML,F,Y,NaN,NaN,NaN,NaN,77.0,Female,ALISA ABRAMOVA,NaN,SU1001,KALININGRAD,MOSCOW,KGD,SVO,2017-01-01,09:15,WMXPVY,4.178367e+15,Y,Moscow,Transit,WMXPVY,False,2017-01-01,09:15,SU1001,1999-12-13,5140 683757,ALISA,ABRAMOVA,R.,Female,Travelgenio,SVO,2017-01-01,12:10,1PC,KGD,YFLXYW,RVML,F,Y,NaN,NaN,NaN,NaN,Moscow
1,NaN,NaN,ULCPQH,False,2017-01-01,09:15,SU1001,NaT,7477 017451,KRISTINA,ARTEMOVA,MARATOVNA,NaN,7.935022e+14,OneTwoTrip,SVO,2017-01-01,12:10,NaN,KGD,YFLXEY,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,YVICRN,False,2017-01-01,09:15,SU1001,NaT,6840 635650,ALEKSANDR,BABUSHKIN,ALEKSANDROVICH,NaN,2.360894e+15,eDreams,SVO,2017-01-01,12:10,1PC,KGD,YSTNUB,NaN,4 FF#FB 483772175,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Moscow,Registered,NaN,False,2017-01-01,09:15,SU1001,1974-07-09,1205 586135,MARK,BEREZIN,A.,Male,8.200536e+15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Moscow
4,NaN,NaN,DBEVCU,False,2017-01-01,09:15,SU1001,1990-05-04,6601 454145,VASILISA,BLINOVA,ILDAROVNA,NaN,7.315197e+15,City.Travel,SVO,2017-01-01,12:10,1PC,KGD,YGRPPY,VGML,0 FF#SU 806242829,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241756,NaN,NaN,VERHMP,False,2018-01-01,09:55,SU6183,NaT,2941 115924,EGOR,LEVIN,SAVELEVICH,NaN,2.038332e+15,City.Travel,ROV,2018-01-01,11:55,NaN,SVO,YSTNII,MOML,#,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
241757,Moscow,None,DWPNNZ,False,2018-01-01,12:45,SU6184,1978-03-30,2191 291628,ANZHELIKA,BLINOVA,MAKAREVNA,Female,6.149777e+15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Moscow
241758,Moscow,Registered,RWUBBA,False,2018-01-01,16:30,SU6286,1973-05-01,2022 416117,ALEKSANDRA,KORCHAGINA,GLEBOVNA,Female,1.650099e+15,NaN,S

In [306]:
joined2['BookingCode'] = joined2.apply(lambda x: my_fun(x, 'BookingCode', 'pnr_2'), axis=1)

In [307]:
joined2['PassengerSex']  = joined2.apply(lambda x: my_fun(x, 'PassengerSex', 'gender_2'), axis=1)

In [308]:
joined2['DepartureAirport']  = joined2.apply(lambda x: my_fun(x, 'DepartureAirport', 'airport_from_2'), axis=1)

In [309]:
joined2['DepartureCity']  = joined2.apply(lambda x: my_fun(x, 'DepartureCity', 'From_2'), axis=1)

In [310]:
joined2['PassengerClass']  = joined2.apply(lambda x: my_fun(x, 'PassengerClass', 'Class_2'), axis=1)

In [311]:
joined2['ArrivalCity']  = joined2.apply(lambda x: my_fun(x, 'ArrivalCity', 'to_2'), axis=1)

In [312]:
joined2['ArrivalAirport'] = joined2.apply(lambda x: my_fun(x, 'ArrivalAirport', 'airport_to_2'), axis=1)

In [313]:
joined2

,ArrivalCity,BaggageState,BookingCode,CodeShare,DepartureDate,DepartureTime,FlightNumber,PassengerBirthDate,PassengerDocument,PassengerFirstName,PassengerLastName,PassengerSecondName,PassengerSex,TicketNumber,AgentInfo,ArrivalAirport,ArrivalDate,ArrivalTime,BaggageCount,DepartureAirport,Fare,Meal,PassengerAdditionalInfo,PassengerClass,ArrivalCountry,DepartureCity,DepartureCountry,PassengerNickName,sequence_2,gender_2,name_2,agency_2,flight_2,From_2,to_2,airport_from_2,airport_to_2,date_2,time_2,pnr_2,TicketNumber_2,Class_2,ArrivalCity_2,BaggageState_2,BookingCode_2,CodeShare_2,DepartureDate_2,DepartureTime_2,FlightNumber_2,PassengerBirthDate_2,PassengerDocument_2,PassengerFirstName_2,PassengerLastName_2,PassengerSecondName_2,PassengerSex_2,AgentInfo_2,ArrivalAirport_2,ArrivalDate_2,ArrivalTime_2,BaggageCount_2,DepartureAirport_2,Fare_2,Meal_2,PassengerAdditionalInfo_2,PassengerClass_2,ArrivalCountry_2,DepartureCity_2,DepartureCountry_2,PassengerNickName_2
0,Moscow,Transit,WMXPVY,False,2017-01-01,09:15,SU1001,1999-12-13,5140 683757,ALISA,ABRAMOVA,R.,Female,4.178367e+15,Travelgenio,SVO,2017-01-01,12:10,1PC,KGD,YFLXYW,RVML,F,Y,NaN,KALININGRAD,NaN,NaN,77.0,Female,ALISA ABRAMOVA,NaN,SU1001,KALININGRAD,MOSCOW,KGD,SVO,2017-01-01,09:15,WMXPVY,4.178367e+15,Y,Moscow,Transit,WMXPVY,False,2017-01-01,09:15,SU1001,1999-12-13,5140 683757,ALISA,ABRAMOVA,R.,Female,Travelgenio,SVO,2017-01-01,12:10,1PC,KGD,YFLXYW,RVML,F,Y,NaN,NaN,NaN,NaN
1,NaN,NaN,ULCPQH,False,2017-01-01,09:15,SU1001,NaT,7477 017451,KRISTINA,ARTEMOVA,MARATOVNA,NaN,7.935022e+14,OneTwoTrip,SVO,2017-01-01,12:10,NaN,KGD,YFLXEY,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,YVICRN,False,2017-01-01,09:15,SU1001,NaT,6840 635650,ALEKSANDR,BABUSHKIN,ALEKSANDROVICH,NaN,2.360894e+15,eDreams,SVO,2017-01-01,12:10,1PC,KGD,YSTNUB,NaN,4 FF#FB 483772175,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Moscow,Registered,NaN,False,2017-01-01,09:15,SU1001,1974-07-09,1205 586135,MARK,BEREZIN,A.,Male,8.200536e+15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,DBEVCU,False,2017-01-01,09:15,SU1001,1990-05-04,6601 454145,VASILISA,BLINOVA,ILDAROVNA,NaN,7.315197e+15,City.Travel,SVO,2017-01-01,12:10,1PC,KGD,YGRPPY,VGML,0 FF#SU 806242829,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241756,NaN,NaN,VERHMP,False,2018-01-01,09:55,SU6183,NaT,2941 115924,EGOR,LEVIN,SAVELEVICH,NaN,2.038332e+15,City.Travel,ROV,2018-01-01,11:55,NaN,SVO,YSTNII,MOML,#,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
241757,Moscow,None,DWPNNZ,False,2018-01-01,12:45,SU6184,1978-03-30,2191 291628,ANZHELIKA,BLINOVA,MAKAREVNA,Female,6.149777e+15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
241758,Moscow,Registered,RWUBBA,False,2018-01-01,16:30,SU6286,1973-05-01,2022 416117,ALEKSANDRA,KORCHAGINA,GLEBOVNA,Female,1.650099e+15,NaN,SVO,2018-01-01,16:05,NaN,PKC,YFLXGK,STML

In [318]:
joined3 = joined2.iloc[: , :28]

In [327]:
joined3['FullName'] = joined2['name']

In [326]:
joined2['name'].notna().sum()

61000

In [329]:
joined.notna().sum(), joined3.notna().sum()

(ArrivalCity                166902
 BaggageState               155152
 BookingCode                197632
 CodeShare                  239717
 DepartureDate              239717
 DepartureTime              239717
 FlightNumber               239717
 PassengerBirthDate         197319
 PassengerDocument          239717
 PassengerFirstName         239717
 PassengerLastName          239717
 PassengerSecondName        239717
 PassengerSex               166902
 TicketNumber               197726
 AgentInfo                  143819
 ArrivalAirport             170078
 ArrivalDate                155762
 ArrivalTime                155762
 BaggageCount                85068
 DepartureAirport           170078
 Fare                       155762
 Meal                        58150
 PassengerAdditionalInfo    130534
 PassengerClass             155762
 ArrivalCountry              37767
 DepartureCity               37767
 DepartureCountry            37767
 PassengerNickName           37767
 dtype: int64,
 Arri

In [331]:
joined3.to_csv('joined2.csv', index = False, encoding = 'CP1251')